In [7]:
import numpy as np
import pandas as pd

In [8]:
# columns = ['Timestamp', 'CAN ID', 'DLC', 'DATA[0]', 'DATA[1]', 'DATA[2]', 'DATA[3]', 'DATA[4]', 'DATA[5]', 'DATA[6]', 'DATA[7]', 'Flag']
dos = pd.read_table('../dataset/DoS_dataset.csv', header=None)
fuzzy = pd.read_table('../dataset/Fuzzy_dataset.csv', header=None)
gear = pd.read_table('../dataset/gear_dataset.csv', header=None)
rpm = pd.read_table('../dataset/RPM_dataset.csv', header=None)

In [9]:
# 0-Normal 1-DoS 2-Fuzzy 3-Spoof 
dos[4] = dos[0].apply(lambda x: 'DoS' if (x.split(',')[-1]=='T') else 'Normal')
fuzzy[4] = fuzzy[0].apply(lambda x: 'Fuzzy' if (x.split(',')[-1]=='T') else 'Normal')
gear[4] = gear[0].apply(lambda x: 'Spoof' if (x.split(',')[-1]=='T') else 'Normal')
rpm[4] = rpm[0].apply(lambda x: 'Spoof' if (x.split(',')[-1]=='T') else 'Normal')

In [ ]:
# DLC
dos[3] = dos[0].apply(lambda x:x.split(',')[2])
fuzzy[3] = fuzzy[0].apply(lambda x:x.split(',')[2])
gear[3] = gear[0].apply(lambda x:x.split(',')[2])
rpm[3] = rpm[0].apply(lambda x:x.split(',')[2])

In [ ]:
# combining all hex bytes of DATA to integer 
dos[2] = dos[0].apply(lambda x:int("0x"+"".join(x.split(',')[3:-1]),16))
fuzzy[2] = fuzzy[0].apply(lambda x:int("0x"+"".join(x.split(',')[3:-1]),16))
gear[2] = gear[0].apply(lambda x:int("0x"+"".join(x.split(',')[3:-1]),16))
rpm[2] = rpm[0].apply(lambda x:int("0x"+"".join(x.split(',')[3:-1]),16))

In [ ]:
# converting hex CAN ID to integer ID
dos[1] = dos[0].apply(lambda x:int("0x"+x.split(',')[1],16))
fuzzy[1] = fuzzy[0].apply(lambda x:int("0x"+x.split(',')[1],16))
gear[1] = gear[0].apply(lambda x:int("0x"+x.split(',')[1],16))
rpm[1] = rpm[0].apply(lambda x:int("0x"+x.split(',')[1],16))

In [ ]:
dos.loc[:,0] = dos[0].apply(lambda x:x.split(',')[0])
fuzzy.loc[:,0] = fuzzy[0].apply(lambda x:x.split(',')[0])
gear.loc[:,0] = gear[0].apply(lambda x:x.split(',')[0])
rpm.loc[:,0] = rpm[0].apply(lambda x:x.split(',')[0])

In [ ]:
data = pd.concat([dos,fuzzy,gear,rpm])

In [ ]:
data.to_csv('../data/intravehicle.csv',index=False)